# DU IA et Santé - Atelier NLP

Dans cet atelier, nous allons voir plusieurs techniques de NLP "moderne", principalement basées sur les transformers et la librairie [huggingface](https://huggingface.co/).

Cet atelier est diviser en trois partie.

Dans la partie 1, nous allons nous intéresser à la partie encodage des transformers (BERT et cie.) afin de mieux comprendre les représentations internes de ceux-ci.

Dans la partie 2, nous interesserons à la partie decodage des transformers (GPT et cie.) et aux méthodes de *prompt engineering* permettant d’améliorer leurs résultats.

Enfin, dans la partie 3, nous allons voir comment méler les deux approches pour construire des assistants personnels.

Mais, avant toute choses, il nous faut installer quelques librairies.

Pour cela, executez la cellule suivante:

In [ ]:
!pip install accelerate jupyter-scatter

Puis, nous allons importer quelques librairies et vérifier que nous accédons bien au GPU (si vous en avez un).

Pour cela exécutez la cellule ci-dessous.

In [ ]:
# Librairies utilitaires
from tqdm.auto import tqdm
import ipywidgets
import gc

# Librairies mathématiques
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
import jscatter

# Librairies ML
import torch
print("cuda available?", str(torch.cuda.is_available()))
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  torch.cuda.set_device(device)
  print("cuda version:", torch.version.cuda)
  print("cuDNN enabled?", torch.backends.cudnn.enabled)
  print("cuDNN version:", torch.backends.cudnn.version())
  print("Device name? ", torch.cuda.get_device_name(torch.cuda.current_device()))

# Initialisation générateurs de nombres aléatoires
SEED = 42
np.random.seed(SEED)
torch.random.manual_seed(SEED)

Si le résultat est : `cuda available? False`, cela signifie que la librairie *torch* n’a pas pu accéder à votre GPU.

Dans le cas contraire (ou si vous n’avez pas de GPU à disposition), nous pouvons passer à la suite.

remarque: si vous utiliser ce GoogleCollab, cliquez sur Exécution > Modifier le type d’exécution et séléctionnnez l’option T4 GPU

## Partie 1 - Encodage et représentations internes

Dans cette partie, nous allons voir comment les transformers, plus spécifiquement les modèles de type BERT, encodent et transforment les textes fournis pour les donner en entrée des modèles. Nous allons aussi étudier les représentations internes de ce modèles, leurs espaces latents, et comment manipuler ces réprésentations.

Pour cela, nous allons nous baser sur le modèle [CamemBERT](https://camembert-model.fr/), entrainé sur des textes en français.

Pour essayer un autre modèle, voir: https://huggingface.co/models

In [ ]:
from transformers import CamembertModel, CamembertTokenizer

model_name = "camembert/camembert-base"

model = CamembertModel.from_pretrained(model_name, device_map=device, torch_dtype="auto")
tokenizer = CamembertTokenizer.from_pretrained(model_name)

### 1.1 Tokenization et encodage

La première étape pour permettre à un modèle de language de traiter du texte est la tokenisation.

Cette étape permet de découper une phrase ou un mot en un ou plusieurs *token* connus du modèle (l’ensemble des tokens connus par un modèle est appelé son "vocabulaire").

Par exemple:

In [ ]:
sentence = "fromage"
tokenized_sentence = tokenizer.tokenize(sentence)
tokenized_sentence

Cependant, un modèle attend en entrée non pas une liste de tokens, mais la liste des id correspondant à ces tokens dans son vocabulaire.

Ainsi, il nous faut encoder les tokens obtenus précédemment en une liste d’entier utilisable par le modèle.

In [ ]:
encoded_sentence = tokenizer.encode(tokenized_sentence, return_tensors="pt")
encoded_sentence

Le résultat ci-dessus présente la liste des ids correspondant aux tokens de notre phrase.

Nous pouvons cependant noter la précense de deux ids supplémentaires (5 et 6), ceux-ci correspondants aux tokens de début et fin de phrase.

Sentez-vous libre de modifier la phrase d’exemple avant de passer à la suite.

### 1.2 Word Embeddings et espaces latents

Maintenant que nous avons comment tokeniser et encoder un texte pour permettre à un modèle de manipuler ce texte, nous allons voir les réprentations internes utilisé par ces modèles pour manipuler ces textes.

Pour cela nous allons utiliser la classe *pipeline* de la librairie *transformers*.

In [12]:
from transformers import pipeline

Cette classe permet d’utiliser nos modèles sur divers problèmes.

Ici, nous allons l’utiliser pour effectuer de la *feature-extraction* et ainsi obtenir les représentations interne des phrases fournies à notre modèle.

In [13]:
pipeline = pipeline('feature-extraction', model=model, tokenizer=tokenizer)

Par exemple:

In [ ]:
data = pipeline(sentence)
pd.DataFrame([data[0][1]])

In [ ]:
pd.DataFrame([np.array(data[0]).mean(axis=0)])

Nous voyons ici, que notre modèle transforme chaque token de notre phrase en vecteurs de taille:

In [ ]:
len(data[0][1])

La taille de ces vecteurs, générallement appellés *embeddings*, determine le nombre de dimensions de l’espace latent de notre modèle.

Chaque vecteur correspondant alors à un point dans cet espace.

Il est alors possible d’avoir un aperçu du vocabulaire d’un modèle dans son espace latent.

Pour cela, il nous faut dans un premier temps recupérer les embeddings de chaque token présent dans le vocabulaire d’un modèle.

In [ ]:
# On récupère le vocabulaire du modèle
vocab = list(tokenizer.get_vocab().keys())

# On encodage le vocabulaire
embedded_vocab = pipeline(vocab, batch_size=32)

# On met le résultat sous un format plus facile à utiliser
embedded_vocab = pd.DataFrame([ev[0][1] for ev in embedded_vocab], index=vocab)

# On affiche le résultat
embedded_vocab

On peut alors récupérer l’embedding d’un token specifique, par exemple:

In [ ]:
embedded_vocab.iloc[[5270]]

In [29]:
from sklearn.decomposition import PCA

In [31]:
pca = PCA(n_components=2)
pca_embedded_vocab = pca.fit_transform(embedded_vocab)

In [32]:
df_pca = pd.DataFrame(data = pca_embedded_vocab, columns = ['PC1', 'PC2'], index=vocab)

In [ ]:
scatter = jscatter.Scatter(data=df_pca, x="PC1", y="PC2")

output = ipywidgets.Output()

@output.capture(clear_output=True)
def selection_change_handler(change):
  display(df_pca.iloc[change.new])

scatter.widget.observe(selection_change_handler, names=["selection"])

ipywidgets.HBox([scatter.show(), output])

Example de visualisation:

https://projector.tensorflow.org/

https://helboukkouri.github.io/embedding-visualization/

Chaque dimension correspond à un "concept" utilisé par le modèle pour différencier les *tokens* entre eux.

Chaque cellule d’un *embedding* correspond alors à la position, allant de -1 à 1, du token associé sur une des dimensions de l’espace latent du modèle.

Nous pouvons, par exemple, visualiser la position d’un token sur chaque dimension à l’aide d’une *heatmap*.

In [ ]:
# On commence par convertir notre embedding dans un format plus adapté
df = pd.DataFrame([data[0][1]])

# Et on lance la représentation (des 10 premières valeurs pour plus de lisibilité)
# sous forme de heatmap
sb.heatmap(df.iloc[:, : 10], vmin=-1.0, vmax=1.0, annot=True, fmt=".2f")

In [ ]:
### Exemples token proches
from sklearn.metrics.pairwise import cosine_similarity

tst = pipeline("crépuscule")
tst = pd.DataFrame([tst[0][1]])

df_tst = pd.DataFrame(cosine_similarity(tst, embedded_vocab)[0], columns=["similarity"], index=vocab)
df_tst = df_tst.sort_values(by=['similarity'], ascending=False)

df_tst.iloc[:10]

Voir:

https://degaucheoudedroite.delemazure.fr/

https://neal.fun/infinite-craft/

In [ ]:
### TODO Exemples math sur vecteur (ROI - HOMME + FEMME)
roi = pipeline("Roi")
roi = pd.DataFrame([roi[0][1]])

homme = pipeline("Homme")
homme = pd.DataFrame([homme[0][1]])

femme = pipeline("Femme")
femme = pd.DataFrame([femme[0][1]])

result = roi - homme + femme

df_tst = pd.DataFrame(cosine_similarity(result, embedded_vocab)[0], columns=["similarity"], index=vocab)
df_tst = df_tst.sort_values(by=['similarity'], ascending=False)

df_tst.iloc[:10]

In [ ]:
roi

In [ ]:
roi - homme + femme

In [ ]:
reine = pipeline("Reine")
reine = pd.DataFrame([reine[0][1]])
reine

In [ ]:
del embedded_vocab
gc.collect()

Pour en savoir plus sur le fonctionnement des embeddings, voir: https://jalammar.github.io/illustrated-word2vec/

### 1.3 Embeddings de documents (TODO)

## Partie 2 - Large modèles de langage et techniques de génération de texte

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
model_name = "microsoft/Phi-3-mini-128k-instruct"

model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-128k-instruct", device_map=device, torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")


In [ ]:
messages = [
    {"role": "user", "content": "Qu'est qu'un cathéter et comment l'utiliser ?"},
]

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [ ]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "do_sample": False,
    # pour utiliser le paramètre "temperature", passer le paramètre "do_sample" à True
    #"temperature": float(SEED)
}

In [ ]:
output = pipe(messages, **generation_args)
print(output[0]['generated_text'])